In [ ]:
import requests as r
import pandas as pd
import os

import sys
import csv
import time
sys.path.append("../../helper_functions")
import clickhouse_utils as ch
sys.path.pop()

In [ ]:
client = ch.connect_to_clickhouse_db() #Default is OPLabs DB
# start_date = '2024-03-01'
trailing_days = 14

chain_names = [
        # 'op',
        'base']

In [ ]:
query = '''
SELECT *
FROM @chain_name@_transactions
WHERE gas_price > 0
        AND block_timestamp >= DATE_TRUNC('day',NOW()) - interval '@trailing_days@ days'
        AND block_timestamp < DATE_TRUNC('day',NOW())
        AND receipt_status = 1

SETTINGS max_execution_time = 50000
'''


In [4]:
for chain_name in chain_names:
        start_time = time.time()
        print(f"Starting processing for {chain_name}")

        q_run = query.replace('@trailing_days@', str(trailing_days))
        q_run = q_run.replace('@chain_name@', chain_name)

        # print(q_run)

        result = client.query_df(q_run)
        int_time = time.time()
        execution_time = int_time - start_time
        print(f"Query for {chain_name} completed in {execution_time:.2f} seconds")

        # start_date_file = start_date.replace('-', '_')
        result.to_csv(f'csv_outputs/{chain_name}_{trailing_days}.csv')

        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Processing for {chain_name} completed in {execution_time:.2f} seconds")
        print("---")

In [ ]:
# print(result.result_rows)